In [1]:
%config Completer.use_jedi = False



In [2]:
import pickle as pkl

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
from prosittransformer.utils import hdf5Loader
from prosittransformer.utils import cleanTapeOutput

In [9]:
prosit_pp = hdf5Loader.from_hdf5("/sdd/berzelius/final_results/delta_0.15/torchResult.hdf5")

Instructions for updating:
Training with HDF5Matrix is not optimized for performance. Instead, we recommend using https://github.com/tensorflow/io to load your HDF5 data into a tf.data Dataset and passing that dataset to Keras.


/home/ekvall/anaconda3/envs/tapeConvert2/lib/python3.6/site-packages/tensorflow/python/keras/utils/io_utils.py:133: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(datapath)


In [10]:
prosit_pp = hdf5Loader.from_hdf5("/sdd/berzelius/final_results/delta_0.15/torchResult.hdf5")
prosit_vanilla = hdf5Loader.from_hdf5("/sdd/PrositToTapeDataConverter/hdf5/hcd/HDF5/prediction_hcd_ho.hdf5")

In [11]:
ALPHABET = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "M(ox)": 21,
}
ALPHABET_S = {integer: char for char, integer in ALPHABET.items()}

In [12]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        X = min(len(lst), i+n)
        yield lst[i:X]

In [13]:
prosit_pp.keys()

dict_keys(['collision_energy', 'collision_energy_aligned_normed', 'collision_energy_normed', 'intensities_pred', 'intensities_raw', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer', 'spectral_angle'])

In [14]:
def getVals(data, key):
    sa_list = list()
    M = data[key]
    for b in tqdm(chunks(M, 1000),total =int(len(M) / 1000)):
        sa_list.append(b)
    all_sa = [s for sa in sa_list for s in sa]
    return np.array(all_sa)

In [15]:
prosit_pp.keys()

dict_keys(['collision_energy', 'collision_energy_aligned_normed', 'collision_energy_normed', 'intensities_pred', 'intensities_raw', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer', 'spectral_angle'])

In [16]:
collision_energy_aligned_normed = getVals(prosit_pp, "collision_energy_aligned_normed")
precursor_charge_onehot = getVals(prosit_pp, "precursor_charge_onehot")


1420it [00:00, 10230.57it/s]                          
1420it [00:00, 8755.08it/s]                         


In [17]:
sequence_integer = getVals(prosit_pp, "sequence_integer")

1420it [00:06, 208.00it/s]                          


In [18]:
pp_intensities_pred = getVals(prosit_pp, "intensities_pred")
pp_intensities_raw = getVals(prosit_pp, "intensities_raw")

1420it [00:23, 60.91it/s]                          
1420it [00:35, 39.69it/s]                          


In [19]:
vanilla_intensities_pred = getVals(prosit_vanilla, "intensities_pred")
vanilla_intensities_raw = getVals(prosit_vanilla, "intensities_raw")

1420it [00:23, 60.03it/s]                          
1420it [00:35, 39.73it/s]                          


In [20]:
X = np.load("/sdd/prosittransformer/data/delta_0.15/predicted_spectra.npy")

In [29]:
df = pd.read_csv("/sdd/prosittransformer/data/prosit_input.csv", sep=",")

In [37]:
from prosittransformer.prositUtils.tensorize import csv
from prosittransformer.prositUtils import sanitize

In [32]:
df = df.head(500)

In [33]:
x = csv(df)

In [34]:
X[:500].shape

(500, 174)

In [35]:
x["intensities_pred"] = X[:500]

In [48]:
X[:500][1]

array([2.52864718e-01, 0.00000000e+00, 0.00000000e+00, 9.00438521e-04,
       0.00000000e+00, 0.00000000e+00, 2.58889258e-01, 0.00000000e+00,
       0.00000000e+00, 1.27201331e+00, 0.00000000e+00, 0.00000000e+00,
       3.22919071e-01, 0.00000000e+00, 0.00000000e+00, 2.76647329e-01,
       0.00000000e+00, 0.00000000e+00, 1.85729593e-01, 0.00000000e+00,
       0.00000000e+00, 1.37496255e-02, 0.00000000e+00, 0.00000000e+00,
       8.45821500e-01, 0.00000000e+00, 0.00000000e+00, 2.73358077e-03,
       0.00000000e+00, 0.00000000e+00, 9.64755893e-01, 0.00000000e+00,
       0.00000000e+00, 7.46197626e-03, 0.00000000e+00, 0.00000000e+00,
       1.29648650e+00, 2.61553600e-02, 3.04961316e-02, 0.00000000e+00,
       0.00000000e+00, 3.30376253e-03, 9.29014385e-01, 0.00000000e+00,
       7.59038776e-02, 0.00000000e+00, 0.00000000e+00, 4.16598886e-01,
       1.03467774e+00, 4.25279588e-01, 2.39662945e-01, 4.71133292e-02,
       1.81184597e-02, 1.88423544e-01, 8.36425483e-01, 5.21491945e-01,
      

In [38]:
data = sanitize.prediction(x)

In [45]:
data["intensities_pred"][1]

array([ 1.9503845e-01,  0.0000000e+00, -1.0000000e+00,  6.9452211e-04,
        0.0000000e+00, -1.0000000e+00,  1.9968528e-01,  0.0000000e+00,
       -1.0000000e+00,  9.8112345e-01,  0.0000000e+00, -1.0000000e+00,
        2.4907245e-01,  0.0000000e+00, -1.0000000e+00,  2.1338235e-01,
        0.0000000e+00, -1.0000000e+00,  1.4325610e-01,  0.0000000e+00,
       -1.0000000e+00,  1.0605298e-02,  0.0000000e+00, -1.0000000e+00,
        6.5239513e-01,  0.0000000e+00, -1.0000000e+00,  2.1084528e-03,
        0.0000000e+00, -1.0000000e+00,  7.4413109e-01,  0.0000000e+00,
       -1.0000000e+00,  5.7555372e-03,  0.0000000e+00, -1.0000000e+00,
        1.0000000e+00,  2.0174032e-02, -1.0000000e+00,  0.0000000e+00,
        0.0000000e+00, -1.0000000e+00,  7.1656311e-01,  0.0000000e+00,
       -1.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.0000000e+00,
       -1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
       -1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
      

In [46]:
pp_intensities_pred[1]

array([ 1.9503841e-01,  0.0000000e+00, -1.0000000e+00,  6.9450581e-04,
        0.0000000e+00, -1.0000000e+00,  1.9968519e-01,  0.0000000e+00,
       -1.0000000e+00,  9.8112339e-01,  0.0000000e+00, -1.0000000e+00,
        2.4907236e-01,  0.0000000e+00, -1.0000000e+00,  2.1338233e-01,
        0.0000000e+00, -1.0000000e+00,  1.4325604e-01,  0.0000000e+00,
       -1.0000000e+00,  1.0605319e-02,  0.0000000e+00, -1.0000000e+00,
        6.5239507e-01,  0.0000000e+00, -1.0000000e+00,  2.1084598e-03,
        0.0000000e+00, -1.0000000e+00,  7.4413103e-01,  0.0000000e+00,
       -1.0000000e+00,  5.7555302e-03,  0.0000000e+00, -1.0000000e+00,
        1.0000000e+00,  2.0174045e-02, -1.0000000e+00,  0.0000000e+00,
        0.0000000e+00, -1.0000000e+00,  7.1656299e-01,  0.0000000e+00,
       -1.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.0000000e+00,
       -1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
       -1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
      

In [ ]:
charge = np.where(precursor_charge_onehot)[1]

In [ ]:
mask = [True if 0 < c and c <3 else False for c in charge]

In [ ]:
mask2 = [True if 21 not in seq else False for seq in sequence_integer]

In [ ]:
sum(mask2)

In [ ]:
MASK = [all([m1, m2]) for m1, m2 in zip(mask, mask2)]

In [ ]:
pp_sa = cleanTapeOutput.masked_spectral_distance(pp_intensities_raw[MASK], pp_intensities_pred[MASK])
1 - np.median(pp_sa)

In [ ]:
vanilla_sa = cleanTapeOutput.masked_spectral_distance(vanilla_intensities_raw[MASK], vanilla_intensities_pred[MASK])
1 - np.median(vanilla_sa)

In [ ]:
modified_sequence = ["".join([ALPHABET_S[s] for s in seq[np.nonzero(seq)]]) for seq in sequence_integer[MASK]]

In [ ]:
charge_val = charge[MASK] + 1
    

In [ ]:
col_energy = collision_energy_aligned_normed[MASK] * 100

In [ ]:
len(col_energy)

In [ ]:
len(charge_val)

In [ ]:
len(modified_sequence)

In [ ]:
df = pd.DataFrame([[p, col, c] for p, c, col in zip(modified_sequence, charge_val, col_energy)], 
             columns=["modified_sequence","collision_energy","precursor_charge"])

In [ ]:
df.to_csv("/sdd/prosittransformer/data/prosit_input.csv", index=False, sep=",")

In [ ]:
'collision_energy_aligned_normed', 'sequence_integer', 'precursor_charge_onehot', 'masses_pred', 'intensities_pred', 'iRT'

In [ ]:
all_results = pkl.load(open("/sdd/prosittransformer/data/all_result.pkl", "rb"))

In [ ]:
all_results.keys()

In [ ]:
all([p1==p2 for p1, p2 in zip(modified_sequence, all_results['modified_sequence'])])

In [ ]:
all_results['modified_sequence']

In [ ]:
all_results['iRT']

In [ ]:
0 < charge[0] and charge[0] <3

In [ ]:
pp_sa = cleanTapeOutput.masked_spectral_distance(pp_intensities_raw, pp_intensities_pred)
1 - np.median(pp_sa)

In [ ]:
vanilla_sa = cleanTapeOutput.masked_spectral_distance(vanilla_intensities_raw, vanilla_intensities_pred)
1 - np.median(vanilla_sa)

In [ ]:
1 - np.median(pp_sa)

In [ ]:
np.median(all_sa)